Exercise for Computer Vision from chapter 3 of this book :  [Machine Learning for Finance](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-finance)

In [1]:
import cv2
import tensorflow as tf

In [2]:
url = "https://solarianprogrammer.com/images/2019/10/21/clouds.jpg"
img_path = tf.keras.utils.get_file('cclouds.jpg', origin=url)
img = cv2.imread(img_path)

In [ ]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow("Over the Clouds",img)
cv2.imshow("Over the Clouds - gray",gray)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
from keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()